> # California School Affordability Analysis

![California](https://www.greatvaluecolleges.net/wp-content/uploads/2021/04/Top-15-Cheapest-Colleges-in-California.jpg)

California is the largest state in the U.S. and has a diverse population, climate, and culture. Housing is notoriously expensive in California. School funding is driven in part by the property taxes of the area that the shcool serves. This leads to better funded schools in neighborhoods with higher housing prices. The purpose of this ananlysis is to compare school performance against housing prices with the goal of finding locations where school performance is high and housing prices are relatively low. In this analysis income is factored into the housing price calculation to get a housing affordability ratio. Areas with high housing prices also tend have higher relative incomes, so I want to control for that. I am personally interested in the outcomes of this analysis and I have a college degree, so I will be considering the median income of college graduates only.

School performance is defined across these dimensions:  
* Greatschools.org rating  

Housing affordability is defined across these dimensions:  
* Median house price
* Median income for college graduates

The metric that we are trying to evaluate could be expressed as the following (higher score is better):  
`school_affordability_index = greatschools_rating / (median_house_price / median_income)`

This analysis will calculate the `school_affordability_index` for each ZIP code in California. Choropleth maps will be at the end of the analysis to help readers visualize the effect of geography on the metric.

*Note: this notebook does not contain code for the intermediary steps of analysis, ie. inspecting the data, troublshooting joins, etc. This notebook is intended to showcase my work and I want to make it enjoyable to read end-to-end. My intention is not to come off hand-wavey, so just know there was a lot of work done that were the building blocks of the code and analysis you find below.*


## Table of Contents
*TOC goes here*

- - - -
## Setup Development Environment
[Mise en place](https://www.wikiwand.com/en/Mise_en_place)

In [1]:
# Package imports
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# Pandas options
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None

# Get current working directory
path = 'C:\\Users\\alexander.beck\\OneDrive - Thermo Fisher Scientific\\Desktop\\CONNOR KINDER REGISTRATION\\school-analysis'
os.chdir(path)
cwd = os.getcwd()
print("Current directory:", cwd)

Current directory: c:\Users\alexander.beck\OneDrive - Thermo Fisher Scientific\Desktop\CONNOR KINDER REGISTRATION\school-analysis


- - - -
## About the source data

### Greatschools.org - School Ratings
Greatschools.org is a non-profit organization that provides school ratings for U.S. schools. Their rating scale is 10 points with 10/10 being the highest. More information about the greatschools.org rating metric can be found here:
* [Greatschools.org School Ratings](https://www.greatschools.org/gk/ratings/)

The following cell contains the code that was used to generate the source data. The greatschools.org API was used to download school rating information. We looped over each city in the Zillow data and sent an API call to request all school ratings data for a city. Finally, we compiled all the API responses into a single data frame for analysis. The cell below is commented out so that the API calls are not repeated each time this notebook is executed.

In [2]:
# Loop over all cities in California and return the available data from the Greatschools API

# from urllib.request import Request, urlopen
# import time

# # Function to call greatschools.org API
# def get_greatschools_data(city:str):
#     """
#     Takes the name of a city as a string.
#     Sends a request to the greatschools API for school data in that city.
#     Cleans the JSON string reponse.
#     Outputs a pandas data frame.
#     """

#     print("Requesting data for:", city)

#     # Clean any spaces in the city name for use in API call
#     city = city.replace(' ', '%20')

#     # Greatschools.org trial API key
#     my_api_key = 'gNzz4LxY4p1GuYFbb8k768NR9xK7MG2U70HMVqrX'

#     go_next_page = True
#     page = 0

#     while go_next_page:

#         # Send API request
#         api_request = Request('https://gs-api.greatschools.org/schools?page=' + str(page) + '&state=ca&city=' + city + '&school_type=public')
#         api_request.add_header('X-API-Key', my_api_key)

#         # Save JSON response
#         api_response = urlopen(api_request).read()

#         # Decode the byte string to UTF-8
#         json_string = api_response.decode('utf-8')

#         # Check for API throttling response
#         if '429 Too Many Requests' in json_string:
#             print('WARNING: API throttling response received:')
#             print(json_string)
#             time.sleep(60)

#         # Drop last character of the JSON string
#         json_string = json_string.rstrip(json_string[-1])

#         # Strip first 11 characters of the JSON string
#         json_string = json_string.lstrip(json_string[0:11])

#         if page > 0:
#             df_response = pd.concat([df_response, pd.read_json(json_string, orient='values')])
#         else:
#             # Load the cleaned JSON string to a data frame
#             df_response = pd.read_json(json_string, orient='values')

#         if len(df_response) == 0:
#             print(city, "not found in greatschools.org data")
#             go_next_page = False
#             continue

#         # Check for pagination
#         if len(df_response) % 50 == 0:
#             page += 1
#         else:
#             go_next_page = False

#     return df_response

# # Get list of city names from Zillow data
# cities = df_zillow_clean['City'].unique().to_list()
# cities.sort()

# # Loop over all cities in California
# df_greatschools_raw = []
# for city in cities:
#     df_city = get_greatschools_data(city)
#     if len(df_city) > 0:
#         if len(df_greatschools_raw) == 0:
#             df_greatschools_raw = df_city.copy()
#         else:
#             df_greatschools_raw = pd.concat([df_greatschools_raw, df_city], ignore_index=True)
#     else:
#         continue

# # Output API data to CSV
# df_greatschools_raw.to_csv('greatschools-california.csv')

In [3]:
# Greatschools.org data files
dir_greatschools = cwd + '\\source-data-files\\greatschools\\'
greatschools_raw_data = 'greatschools-california.csv'

# Read the greatschools.org data
df_greatschools_raw = pd.read_csv(dir_greatschools + greatschools_raw_data, index_col=0)

# Drop the 

print("Imported Greatschools.org data successfully")

Imported Greatschools.org data successfully


### Zillow - Home Price Index Data
Zillow is a real estate tech company that provides myriad services relating to housing. They open source some of their data, including housing prices across regions in the U.S. Zillow uses the ZHVI metric for their housing data. More information about ZHVI and Zillow data can be found here:  
* [ZHVI (**Z**illow **H**ome **V**alue **I**ndex)](https://www.zillow.com/research/zhvi-methodology-2019-highlights-26221/)

In [4]:
# Zillow data files
dir_zillow = cwd + '\\source-data-files\\zillow\\'
zillow_raw_data = 'Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'

# Read Zillow data
df_zillow_raw_data = pd.read_csv(dir_zillow + zillow_raw_data)

print("Imported Zillow data successfully")

Imported Zillow data successfully


### U.S. Census Bureau - Household Income Data
The U.S. Census Bureau publishes an immense amount of aggregated demographic data. The specific dataset of interest for this analysis is apart of the American Community Survey (ACS) for the year 2020. The exact dataset used can be found here:  
* [B20004 | MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2020 INFLATION-ADJUSTED DOLLARS) BY SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER](https://data.census.gov/cedsci/table?q=B20004&t=Educational%20Attainment%3AIncome%20and%20Earnings&g=0400000US06%248600000)  
  
Below are links to the survey's data collection methodolgies and other technical documentation:
* [Data Collection Methodologies](https://www.census.gov/acs/www/methodology/sample_size_and_data_quality/)
* [Technical Documentation](https://www.census.gov/programs-surveys/acs/technical-documentation/code-lists.html)  
  
This analysis is of interest to me personally, therefore I am specifically looking at the median income for college graduates for each ZIP code in California. With that being said, I will also include household incomes for the general population to allow for more generlized conclusions to be drawn.

In [5]:
# Census Median Income data files
dir_census = cwd + '\\source-data-files\\us-census-bureau\\'
income_zip_data = 'income-data-by-zip-california.csv'
income_edu_zip_data = 'income-by-zip-by-education-by-sex-california.csv'

# Read Census Bureau data
df_income_raw = pd.read_csv(dir_census + income_zip_data)
df_income_edu_raw = pd.read_csv(dir_census + income_edu_zip_data)

print("Imported U.S. Census Bureau data successfully")

Imported U.S. Census Bureau data successfully


### Metadata overview

In [6]:
# Build dataframe for storing metadata
metadata_dict = {'Greatschools Ratings': ['Greatschools.org', df_greatschools_raw],
                 'Zillow Housing Data': ['Zillow', df_zillow_raw_data],
                 'Income by ZIP': ['U.S. Census Bureau', df_income_raw],
                 'Income by ZIP and Education': ['U.S. Census Bureau', df_income_edu_raw]}
                 
raw_data_names = metadata_dict.keys()
sources = [value[0] for key, value in metadata_dict.items()]
dataframes = [value[1] for key, value in metadata_dict.items()]
records = [df.shape[0] for df in dataframes]
columns = [df.shape[1] for df in dataframes]
mem_usage = [df.memory_usage().sum()/1024**2 for df in dataframes]

df_metadata = pd.DataFrame({'Dataset': raw_data_names,
                            'Source': sources,
                            'Records': records,
                            'Columns': columns,
                            'Memory Usage (MB)': mem_usage})
df_metadata

,Dataset,Source,Records,Columns,Memory Usage (MB)
0,Greatschools Ratings,Greatschools.org,9047,25,1.79
1,Zillow Housing Data,Zillow,30523,275,64.04
2,Income by ZIP,U.S. Census Bureau,1771,242,3.27
3,Income by ZIP and Education,U.S. Census Bureau,1770,38,0.51


- - - -
## Data Quality

### Missing values and duplicated records

In [7]:
def data_quality_report(df_metadata=df_metadata, dataframes=dataframes):
    """
    Take the list of source dataframes and their metadata as input. 
    Calculate total data points, missing values, and the proportion of missing values to total values.
    Output data quality report as pandas dataframe.
    """
    # Make a new dataframe for analyzing data quality
    df_data_quality = df_metadata[['Dataset','Records','Columns']]

    # Get total number of data points
    df_data_quality['Total Data Points'] = df_data_quality['Records'] * df_data_quality['Columns']

    # Get total number of null values
    df_data_quality['Null Values'] = [df.isna().sum().sum() for df in dataframes]

    # Get total number of `*` values
    df_data_quality['Other Missing Values'] = [int(df.apply(pd.Series.value_counts).loc['*'].sum()) if df.isin(['*']).sum().sum() > 0 else 0 for df in dataframes]

    # Get total number of null and missing values
    df_data_quality['Missing Values'] = df_data_quality['Null Values'] + df_data_quality['Other Missing Values']

    # Calculate rate of null and missing values
    df_data_quality['Missing Value Rate'] = df_data_quality['Missing Values'] / df_data_quality['Total Data Points']

    # Drop columns used for calculation
    df_data_quality.drop(['Null Values','Other Missing Values'], axis=1, inplace=True)

    # Calculate number of duplicated records
    df_data_quality['Duplicate Records'] = [df.duplicated().sum() for df in dataframes]

    return df_data_quality

# Execute data quality report
df_data_quality = data_quality_report(df_metadata, dataframes)

# Show missing value analysis
df_data_quality.sort_values(by='Missing Value Rate', ascending=False).style.background_gradient(cmap='PuBu')

,Dataset,Records,Columns,Total Data Points,Missing Values,Missing Value Rate,Duplicate Records
1,Zillow Housing Data,30523,275,8393825,1749973,0.208483,0
0,Greatschools Ratings,9047,25,226175,7550,0.033381,0
2,Income by ZIP,1771,242,428582,0,0.000000,0
3,Income by ZIP and Education,1770,38,67260,0,0.000000,0


- - - -
## Data Preparation

### Greatschools Data

#### Cleaning the Greatschools data
Cleaning steps:
1. Drop uneccesary columns
2. Handle missing values
3. Assign data types
4. Reformat School Name values

In [8]:
def clean_greatschools_data(data_table):
    """
    Take the raw data extracted from the Greatschools API and perform cleaning operations for this analysis.
    Output the cleaned dataframe.
    """
    # Get categorical columns
    cat_cols = ['universal-id','name','type','level-codes','city','state','zip','county','district-name','district-id','year']

    # Get float columns
    float_cols = ['rating']

    # Get the geographical columns
    geo_cols = ['lat','lon']

    # Get the object columns
    obj_cols = ['school-summary','overview-url']

    # Drop unnecessary columns
    cols = cat_cols + float_cols + geo_cols + obj_cols
    df_clean = data_table[cols]

    # Drop records with no school rating
    df_clean = df_clean[~df_clean['rating'].isna()]

    # Assign data dtypes
    df_clean[cat_cols] = df_clean[cat_cols].astype('object')
    df_clean[float_cols] = df_clean[float_cols].astype('float32')
    df_clean[geo_cols] = df_clean[geo_cols].astype('float32')
    df_clean[obj_cols] = df_clean[obj_cols].astype('object')

    # Drop the 'School' at the end of the school name values for joining to the test scores data
    df_clean['name'] = df_clean['name'].str.replace(' School$', '', regex=True)

    return df_clean

# Clean the Greatschools.org data
df_greatschools_clean = clean_greatschools_data(df_greatschools_raw)

print("Greatschools.org Data cleaned successfully")

Greatschools.org Data cleaned successfully


#### Viewing the Greatschools data

In [9]:
df_greatschools_clean.head(3)

,universal-id,name,type,level-codes,city,state,zip,county,district-name,district-id,year,rating,lat,lon,school-summary,overview-url
2,606561,Houston,public,"e,m",Acampo,CA,95220,San Joaquin County,Lodi Unified School District,752,"2,020.00",3.00,38.17,-121.26,"Houston School, a public school located in Aca...",https://www.greatschools.org/california/acampo...
3,606612,Oak View Elementary,public,"e,m",Acampo,CA,95220,San Joaquin County,Oak View Union Elementary School District,757,"2,020.00",6.00,38.22,-121.23,"Oak View Elementary School, a public school lo...",https://www.greatschools.org/california/acampo...
6,601454,High Desert,public,"e,m",Acton,CA,93510,Los Angeles County,Acton-Agua Dulce Unified School District,249,"2,020.00",6.00,34.49,-118.19,"High Desert School, a public school located in...",https://www.greatschools.org/california/acton/...


In [10]:
#df_greatschools_clean = df_greatschools_raw.copy()
df_greatschools_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7167 entries, 2 to 9049
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   universal-id    7167 non-null   object 
 1   name            7167 non-null   object 
 2   type            7167 non-null   object 
 3   level-codes     7167 non-null   object 
 4   city            7167 non-null   object 
 5   state           7167 non-null   object 
 6   zip             7167 non-null   object 
 7   county          7166 non-null   object 
 8   district-name   7167 non-null   object 
 9   district-id     7167 non-null   object 
 10  year            7167 non-null   object 
 11  rating          7167 non-null   float32
 12  lat             7167 non-null   float32
 13  lon             7167 non-null   float32
 14  school-summary  7167 non-null   object 
 15  overview-url    7167 non-null   object 
dtypes: float32(3), object(13)
memory usage: 867.9+ KB


### Zillow Home Price Index Data

#### Cleaning the Zillow data
The data from Zillow is fairly clean as-is. The data comes in a time-series format, but we aren't interested in time-series for this analysis. We will only keep the most recent housing prices in the data and drop all the other time indexes and other unnecessary columnsl.

In [11]:
def clean_zillow_data(data_table):
    """
    Takes input data from Zillow and perform cleaning operations.
    Output cleaned data frame.
    """
    # Select categorical columns
    cat_cols = ['SizeRank','RegionName','State','City','Metro','CountyName']
    
    # Get the last column in the data set (most recent price data)
    float_col = data_table.columns.to_list()[-1]

    # Cleaned feature set
    cols = cat_cols
    cols.append(float_col)
    df_clean = data_table[cols]

    # Get only California data
    df_clean = df_clean[df_clean['State'] == 'CA']

    # Insert a record for median home price in the entire state
    df_clean.loc[-1] = ['', '', 'CA', '', '', '', df_clean[float_col].mean()]
    df_clean.reset_index()

    # Define data types
    df_clean[cat_cols] = df_clean[cat_cols].astype('object')
    df_clean[float_col] = df_clean[float_col].astype('float32')

    # Rename columns
    df_clean.rename(columns={float_col: 'MedianHomePrice', 'RegionName': 'ZipCode', 'City': 'CityName',
                                        'State': 'StateName'}, inplace=True)

    return df_clean

# Clean the Zillow Home Price Index data
df_zillow_clean = clean_zillow_data(df_zillow_raw_data)

print("Zillow Data cleaned successfully")

Zillow Data cleaned successfully


#### Viewing the Zillow data

In [12]:
# Preview the Zillow Home Value Index data
df_zillow_clean.tail()

,SizeRank,ZipCode,StateName,CityName,Metro,CountyName,MedianHomePrice
30465,34322,95375,CA,Strawberry,Sonora,Tuolumne County,"405,091.00"
30484,34430,95229,CA,Vallecito,NaN,Calaveras County,"384,331.00"
30495,34430,95721,CA,Twin Bridges,Sacramento--Roseville--Arden-Arcade,El Dorado County,"442,415.00"
30509,34430,93282,CA,Tulare,Visalia-Porterville,Tulare County,"234,713.00"
-1,,,CA,,,,"831,724.31"


In [13]:
# Show metadata
df_zillow_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1708 entries, 13 to -1
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SizeRank         1708 non-null   object 
 1   ZipCode          1708 non-null   object 
 2   StateName        1708 non-null   object 
 3   CityName         1708 non-null   object 
 4   Metro            1567 non-null   object 
 5   CountyName       1708 non-null   object 
 6   MedianHomePrice  1708 non-null   float32
dtypes: float32(1), object(6)
memory usage: 100.1+ KB


### U.S. Census Income Data

#### Cleaning the Census data

In [14]:
def clean_census_data(data_table):
    """
    
    """
    # Drop margin-of-error and male/female columns
    cols = [col for col in data_table.columns if col[-1] != 'M'][0:6]
    cols.insert(0, 'NAME')
    df_clean = data_table[cols]

    # Replace the column header codes with descriptive names
    df_clean.drop(0, inplace=True)
    df_clean.rename(columns={'NAME': 'zip',
                             'B20004_001E': 'total',
                             'B20004_002E': 'less_than_hs_grad',
                             'B20004_003E': 'hs_grad',
                             'B20004_004E': 'some_college',
                             'B20004_005E': 'bachelors_degree',
                             'B20004_006E': 'graduate_degree'},
                    inplace=True)

    # Clean up the ZIP code values
    df_clean['zip'] = df_clean['zip'].str.split(' ').str[-1]

    # Get float columns
    float_cols = ['total','less_than_hs_grad','hs_grad','some_college','bachelors_degree','graduate_degree']

    # Fill missing values (will impute them later)
    df_clean[float_cols] = df_clean[float_cols].replace(['-','2,500-','250,000+'], ['0','0','250000'])
    
    # Assign data types
    df_clean[float_cols] = df_clean[float_cols].astype('float32')

    # Drop records with no income amounts
    df_clean = df_clean[df_clean[float_cols].sum(axis=1) != 0]

    return df_clean

# Clean the U.S. Census data for income by zip and education level
df_income_edu_clean = clean_census_data(df_income_edu_raw)

df_income_edu_clean.head()

,zip,total,less_than_hs_grad,hs_grad,some_college,bachelors_degree,graduate_degree
1,89010,"26,563.00",0.00,"33,750.00","25,000.00","74,375.00",0.00
2,89019,"36,667.00","17,436.00",0.00,"53,309.00",0.00,"156,000.00"
3,89060,"41,471.00","42,609.00","40,322.00","42,375.00",0.00,"60,595.00"
4,89061,"34,705.00","63,036.00","39,861.00","30,917.00","58,105.00","17,942.00"
5,89439,"42,431.00",0.00,"34,176.00","32,083.00","54,618.00","118,819.00"


##### Imputing missing values in the census data
For ZIP codes with small sample sizes, the Census Bureau's data does not calculate a median income. I will impute these uncalculated values using the mean percent change in income for each education level. For example, if the sample size of `graduate_degree` residents of a ZIP code was too small for the Census Bureau to estimate a median income, then I will take the `bachelors_degree` median income and multiply it by `1 + 0.3831` to represent the average increase in income of 35% for residents with a graduate or professional degree. If there are no data points for either `bachelors_degree` or `graduate_degree` then I will use the overall median income (`total`).

In [15]:
# Get only records with no missing values for bachelors and grad degrees
df_income_edu_no_missing = df_income_edu_clean[(
                                                (df_income_edu_clean['total']!=0) & 
                                                (df_income_edu_clean['bachelors_degree']!=0) & 
                                                (df_income_edu_clean['graduate_degree']!=0)
                                               )][['total','bachelors_degree','graduate_degree']]

# Calculate mean incomes for the general population, those with bachelors degrees and those with grad degrees
general_population_income = df_income_edu_no_missing['total'].mean()
bachelors_degree_income = df_income_edu_no_missing['bachelors_degree'].mean()
graduate_degree_income = df_income_edu_no_missing['graduate_degree'].mean()

# Calculate the percentage change in income between general population, bachelors and graduate degrees
s_pct_change = pd.Series([general_population_income, bachelors_degree_income, graduate_degree_income])
bachelor_degree_pct_change = s_pct_change.pct_change()[1]
grad_degree_pct_change = s_pct_change.pct_change()[2]

print("Mean income for the general population:", "{:,.0f}".format(general_population_income))
print("Mean income for those with Bachelor's degrees:", "{:,.0f}".format(bachelors_degree_income))
print("Mean income for those with Graduate or Professional degrees:", "{:,.0f}".format(graduate_degree_income))
print("Mean percentage increase in income for those with Bachelor's degrees vs. general population:", "{:.2%}".format(bachelor_degree_pct_change))
print("Mean percentage increase in income for those with Graduate or Professional degrees vs. Bachelor's degrees:", "{:.2%}".format(grad_degree_pct_change))



Mean income for the general population: 50,526
Mean income for those with Bachelor's degrees: 63,761
Mean income for those with Graduate or Professional degrees: 85,381
Mean percentage increase in income for those with Bachelor's degrees vs. general population: 26.19%
Mean percentage increase in income for those with Graduate or Professional degrees vs. Bachelor's degrees: 33.91%


In [16]:
# Make columns to use for income estimation
df_income_edu_clean['grad_degree_pct_change'] = 1 + grad_degree_pct_change
df_income_edu_clean['bachelors_degree_pct_change'] = 1 + bachelor_degree_pct_change

# Estimate bachelors degree income using graduate degree income
df_income_edu_clean.loc[df_income_edu_clean['bachelors_degree'] == 0, 'bachelors_degree'] = (
    df_income_edu_clean['graduate_degree'] / df_income_edu_clean['grad_degree_pct_change']
    ).round(0)

# Estimate bachelors degree income using general population income
df_income_edu_clean.loc[df_income_edu_clean['bachelors_degree'] == 0, 'bachelors_degree'] = (
    df_income_edu_clean['total'] * df_income_edu_clean['bachelors_degree_pct_change']
    ).round(0)

# Estimate graduate degree income
df_income_edu_clean.loc[df_income_edu_clean['graduate_degree'] == 0, 'graduate_degree'] = (
    df_income_edu_clean['bachelors_degree'] * df_income_edu_clean['grad_degree_pct_change']
    ).round(0)

# Drop columns used for income estimation
df_income_edu_clean.drop(['grad_degree_pct_change','bachelors_degree_pct_change'], axis=1, inplace=True)

print("U.S. Census data cleaned successfully")

U.S. Census data cleaned successfully


#### Viewing the Census data

In [17]:
df_income_edu_clean.head()

,zip,total,less_than_hs_grad,hs_grad,some_college,bachelors_degree,graduate_degree
1,89010,"26,563.00",0.00,"33,750.00","25,000.00","74,375.00","99,594.00"
2,89019,"36,667.00","17,436.00",0.00,"53,309.00","116,498.00","156,000.00"
3,89060,"41,471.00","42,609.00","40,322.00","42,375.00","45,251.00","60,595.00"
4,89061,"34,705.00","63,036.00","39,861.00","30,917.00","58,105.00","17,942.00"
5,89439,"42,431.00",0.00,"34,176.00","32,083.00","54,618.00","118,819.00"


In [18]:
df_income_edu_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1641 entries, 1 to 1769
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   zip                1641 non-null   object 
 1   total              1641 non-null   float32
 2   less_than_hs_grad  1641 non-null   float32
 3   hs_grad            1641 non-null   float32
 4   some_college       1641 non-null   float32
 5   bachelors_degree   1641 non-null   float32
 6   graduate_degree    1641 non-null   float32
dtypes: float32(6), object(1)
memory usage: 128.6+ KB


### Data quality after cleaning

In [19]:
# Build dataframe for storing metadata
clean_metadata_dict = {'Greatschools Ratings': ['Greatschools.org', df_greatschools_clean],
                       'Zillow Housing Data': ['Zillow', df_zillow_clean],
                       'Income by ZIP and Education': ['U.S. Census Bureau', df_income_edu_clean]}
                 
clean_data_names = clean_metadata_dict.keys()
sources = [value[0] for key, value in clean_metadata_dict.items()]
clean_dataframes = [value[1] for key, value in clean_metadata_dict.items()]
records = [df.shape[0] for df in clean_dataframes]
columns = [df.shape[1] for df in clean_dataframes]
mem_usage = [df.memory_usage().sum()/1024**2 for df in clean_dataframes]

df_clean_metadata = pd.DataFrame({'Dataset': clean_data_names,
                            'Source': sources,
                            'Records': records,
                            'Columns': columns,
                            'Memory Usage (MB)': mem_usage})
df_clean_metadata

,Dataset,Source,Records,Columns,Memory Usage (MB)
0,Greatschools Ratings,Greatschools.org,7167,16,0.85
1,Zillow Housing Data,Zillow,1708,7,0.10
2,Income by ZIP and Education,U.S. Census Bureau,1641,7,0.13


#### Comparison of the data before and after cleaning
Below we can see that the data cleaning operations had a great affect on the missing values in the data. Preliminary joining of the source data also reduced the number of dataframes I'm working with. The remaining missing values are intentional, as there are lower-level dimensions in some of the data that have to be missing when a record is used for aggregation. Further along in the notebook I can choose the aggregated records for a different level of analysis.

In [20]:
# Show data quality report on pre-cleaned data
df_data_quality.sort_values(by='Missing Value Rate', ascending=False,).style.background_gradient(cmap='PuBu')

,Dataset,Records,Columns,Total Data Points,Missing Values,Missing Value Rate,Duplicate Records
1,Zillow Housing Data,30523,275,8393825,1749973,0.208483,0
0,Greatschools Ratings,9047,25,226175,7550,0.033381,0
2,Income by ZIP,1771,242,428582,0,0.000000,0
3,Income by ZIP and Education,1770,38,67260,0,0.000000,0


In [21]:
# Execute data quality report on cleaned data
df_clean_data_quality = data_quality_report(df_clean_metadata, clean_dataframes)

# Show data quality report on cleaned data
df_clean_data_quality.sort_values(by='Missing Value Rate', ascending=False).style.background_gradient(cmap='PuBu')

,Dataset,Records,Columns,Total Data Points,Missing Values,Missing Value Rate,Duplicate Records
1,Zillow Housing Data,1708,7,11956,141,0.011793,0
0,Greatschools Ratings,7167,16,114672,1,0.000009,0
2,Income by ZIP and Education,1641,7,11487,0,0.000000,0


## Joining the data
Here I will join the cleaned data sets into one big data set for the main analysis.

In [22]:
# Force the joining columns to string data types
df_greatschools_clean[['name','zip']] = df_greatschools_clean[['name','zip']].astype(str)
df_zillow_clean['ZipCode'] = df_zillow_clean['ZipCode'].astype(str)
df_income_edu_clean['zip'] = df_income_edu_clean['zip'].astype(str)

### First join condition
`df_zillow_clean['ZipCode'] = df_income_edu_clean['zip']`  
After joining the data on the zip code, I will calculate the `affordability_ratio` which is expressed as:  
`affordability_ratio = median_home_price / median_income_bachelors_degree`  

In [23]:
# Join the Zillow Home Price data with the U.S. Census Income data
df_final = pd.merge(left=df_zillow_clean, right=df_income_edu_clean, 
                    how='outer', left_on='ZipCode', right_on='zip')

# Calculate the affordability metric by dividing home price by income
df_final['affordability_ratio'] = df_final['MedianHomePrice'] / df_final['bachelors_degree']

# Show the result
print(f"Merged data set contains {len(df_final)} rows")
df_final.head()

Merged data set contains 1751 rows


,SizeRank,ZipCode,StateName,CityName,Metro,CountyName,MedianHomePrice,zip,total,less_than_hs_grad,hs_grad,some_college,bachelors_degree,graduate_degree,affordability_ratio
0,13,94109,CA,San Francisco,San Francisco-Oakland-Hayward,San Francisco County,"1,225,694.00",94109,"83,206.00","23,346.00","36,297.00","45,838.00","98,961.00","107,428.00",12.39
1,22,90250,CA,Hawthorne,Los Angeles-Long Beach-Anaheim,Los Angeles County,"852,909.00",90250,"35,779.00","23,952.00","30,949.00","38,374.00","53,537.00","60,783.00",15.93
2,40,94565,CA,Pittsburg,San Francisco-Oakland-Hayward,Contra Costa County,"800,936.00",94565,"40,694.00","26,645.00","33,884.00","45,102.00","61,171.00","69,571.00",13.09
3,44,90046,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,"1,673,691.00",90046,"55,080.00",0.00,"31,159.00","39,914.00","63,757.00","83,459.00",26.25
4,88,94501,CA,Alameda,San Francisco-Oakland-Hayward,Alameda County,"1,394,612.00",94501,"63,005.00","28,750.00","34,269.00","43,608.00","75,473.00","99,904.00",18.48


### Second join condition
`df_final['Zip Code'] = df_greatschools_clean['zip']`  
After joining the data on zip code I will calculate the `school_affordability_index` which is expressed as:  
`school_affordability_index = greatschools_rating / affordability_ratio`

In [24]:
# Group the greatschools data by zip code and get the mean rating
#df_greatschools_by_zip = df_greatschools_clean.groupby('zip').mean()['rating']

# Join the [home price + income] data with mean Greatschools rating by zip code
df_final = pd.merge(left=df_final, right=df_greatschools_clean, 
                    how='outer', left_on='zip', right_on='zip')

# Calculate the school affordability index by dividing the greatschools rating by the affordability ratio
df_final['school_affordability_index'] = df_final['rating'] / df_final['affordability_ratio']

# Show the result
print(f"Merged data set contains {len(df_final)} rows")
df_final.head(2)

Merged data set contains 7695 rows


,SizeRank,ZipCode,StateName,CityName,Metro,CountyName,MedianHomePrice,zip,total,less_than_hs_grad,hs_grad,some_college,bachelors_degree,graduate_degree,affordability_ratio,universal-id,name,type,level-codes,city,state,county,district-name,district-id,year,rating,lat,lon,school-summary,overview-url,school_affordability_index
0,13,94109,CA,San Francisco,San Francisco-Oakland-Hayward,San Francisco County,"1,225,694.00",94109,"83,206.00","23,346.00","36,297.00","45,838.00","98,961.00","107,428.00",12.39,606369,Galileo High,public,h,San Francisco,CA,San Francisco County,San Francisco Unified School District,717,"2,020.00",8.00,37.80,-122.42,"Galileo High School, a public school located i...",https://www.greatschools.org/california/san-fr...,0.65
1,13,94109,CA,San Francisco,San Francisco-Oakland-Hayward,San Francisco County,"1,225,694.00",94109,"83,206.00","23,346.00","36,297.00","45,838.00","98,961.00","107,428.00",12.39,606424,Redding Elementary,public,e,San Francisco,CA,San Francisco County,San Francisco Unified School District,717,"2,020.00",7.00,37.79,-122.42,"Redding Elementary School, a public school loc...",https://www.greatschools.org/california/san-fr...,0.57


## Final Cleaning
Quickly perform some final cleaning operations on the joined data set to make working with it easeier.  
1. Rename columns to a consistant format
2. Drop redundant columns  
3. Drop records with incomplete data
4. One-hot encode the `level_codes` column
5. Assign data dtypes

In [25]:
# Reformat all columns to snake_case
columns_snake_case = {col: col.lower().replace('-','_') for col in df_final.columns}
df_final.rename(columns=columns_snake_case, inplace=True)

# Rename columns from the Zillow data set to have underscores
df_final.rename(columns={'sizerank': 'size_rank',
                         'cityname': 'city_name',
                         'zipcode': 'zip_code',
                         'countyname': 'county_name',
                         'statename': 'state_name',
                         'medianhomeprice': 'median_home_price',
                         'name': 'school_name',
                         'type': 'school_type'},
                inplace=True)

# Set the zip code from the Zillow data set to the index and drop other zip code columns
df_final.set_index('zip_code', inplace=True)
df_final.drop(['zip','city','state','county'], axis=1, inplace=True)

# Drop rows with null index (`zip_code`)
df_final = df_final[~(df_final.index.isna())]

# One-hot encode the `level_codes` column
df_final['level_codes'].fillna('0', inplace=True)
df_final[['pre','elementary','middle','high']] = 0
df_final.loc[df_final['level_codes'].str.contains('p'), 'pre'] = 1
df_final.loc[df_final['level_codes'].str.contains('e'), 'elementary'] = 1
df_final.loc[df_final['level_codes'].str.contains('m'), 'middle'] = 1
df_final.loc[df_final['level_codes'].str.contains('m'), 'high'] = 1

# Get categorical columns
cat_cols = ['state_name','city_name','metro','county_name','universal_id','school_name','school_type',
            'pre','elementary','middle','high','district_name','district_id', 'year']

# Get float columns
float_cols = ['median_home_price','total','less_than_hs_grad','hs_grad','some_college',
              'bachelors_degree','graduate_degree','affordability_ratio','rating','school_affordability_index']

# Geo Pandas columns
geo_cols = ['lat','lon']

# Get final feature set
cols = cat_cols + float_cols + geo_cols
df_final = df_final[cols]

# Assign data types
df_final[float_cols] = df_final[float_cols].astype('float32')
df_final[cat_cols] = df_final[cat_cols].astype('category')
df_final[geo_cols] = df_final[geo_cols].astype('float32')

df_final.head(3)

,state_name,city_name,metro,county_name,universal_id,school_name,school_type,pre,elementary,middle,high,district_name,district_id,year,median_home_price,total,less_than_hs_grad,hs_grad,some_college,bachelors_degree,graduate_degree,affordability_ratio,rating,school_affordability_index,lat,lon
zip_code,,,,,,,,,,,,,,,,,,,,,,,,,,
94109,CA,San Francisco,San Francisco-Oakland-Hayward,San Francisco County,606369,Galileo High,public,0,0,0,0,San Francisco Unified School District,717,"2,020.00","1,225,694.00","83,206.00","23,346.00","36,297.00","45,838.00","98,961.00","107,428.00",12.39,8.00,0.65,37.80,-122.42
94109,CA,San Francisco,San Francisco-Oakland-Hayward,San Francisco County,606424,Redding Elementary,public,0,1,0,0,San Francisco Unified School District,717,"2,020.00","1,225,694.00","83,206.00","23,346.00","36,297.00","45,838.00","98,961.00","107,428.00",12.39,7.00,0.57,37.79,-122.42
94109,CA,San Francisco,San Francisco-Oakland-Hayward,San Francisco County,606434,Spring Valley Elementary,public,0,1,0,0,San Francisco Unified School District,717,"2,020.00","1,225,694.00","83,206.00","23,346.00","36,297.00","45,838.00","98,961.00","107,428.00",12.39,6.00,0.48,37.79,-122.42


In [26]:
df_final[df_final.index=='']

,state_name,city_name,metro,county_name,universal_id,school_name,school_type,pre,elementary,middle,high,district_name,district_id,year,median_home_price,total,less_than_hs_grad,hs_grad,some_college,bachelors_degree,graduate_degree,affordability_ratio,rating,school_affordability_index,lat,lon
zip_code,,,,,,,,,,,,,,,,,,,,,,,,,,
,CA,,,,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,"831,724.31",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Exploratory Data Analysis
Let's look at some descriptive statistics about the data set and the `school_afforability_index`

In [27]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7644 entries, 94109 to 96132
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   state_name                  7644 non-null   category
 1   city_name                   7644 non-null   category
 2   metro                       7438 non-null   category
 3   county_name                 7644 non-null   category
 4   universal_id                7159 non-null   category
 5   school_name                 7159 non-null   category
 6   school_type                 7159 non-null   category
 7   pre                         7644 non-null   category
 8   elementary                  7644 non-null   category
 9   middle                      7644 non-null   category
 10  high                        7644 non-null   category
 11  district_name               7159 non-null   category
 12  district_id                 7159 non-null   category
 13  year              

In [28]:
### REMOVE NULL RATINGS. TAKE AWAY THIS CELL WHEN THE FULL GREATSCHOOLS DATASET IS AVAILABLE ###
df_final = df_final[~df_final['rating'].isna()]

## Exploring the greatschools `rating` 

In [29]:
# Top 10 cities by Greatschools rating
df_rating = df_final.groupby(['county_name','city_name'], as_index=False, )['rating'].mean()
df_rating.sort_values(by='rating', ascending=False).head(10).style.background_gradient(cmap='PuBu_r')

,county_name,city_name,rating
31661,Placer County,Granite Bay,9.800000
19998,Los Angeles County,San Marino,9.250000
7533,Contra Costa County,Lafayette,9.166667
7728,Contra Costa County,Orinda,9.166667
30819,Orange County,Los Alamitos,9.125000
22125,Marin County,Tiburon,9.000000
20158,Los Angeles County,West Hollywood,9.000000
53778,Trinity County,Douglas City,9.000000
30506,Orange County,Coto de Caza,9.000000
7666,Contra Costa County,Moraga,9.000000


## Exploring the calculated `school_affordability_index` 

In [30]:
# Top 10 cities by Greatschools rating
df_sai = df_final.groupby(['county_name','city_name'], as_index=False, )['school_affordability_index'].mean()
df_sai = df_sai[df_sai['school_affordability_index']!=0]
df_sai.sort_values(by='school_affordability_index', ascending=True).head(20).style.background_gradient(cmap='PuBu_r')

,county_name,city_name,school_affordability_index
39684,San Joaquin County,Farmington,0.046092
44239,Santa Clara County,San Martin,0.068566
44308,Santa Clara County,Stanford,0.070924
41453,San Mateo County,Atherton,0.080149
22129,Marin County,Tomales,0.117119
31752,Placer County,Kings Beach,0.125732
28075,Monterey County,San Lucas,0.135071
21923,Marin County,Point Reyes Station,0.138213
49830,Sonoma County,Glen Ellen,0.143014
43011,Santa Barbara County,Montecito,0.149497


## Exploring the `affordability_ratio`

In [31]:
df = df_final[df_final['median_home_price'] > 200000]
df = df.groupby(['county_name'], as_index=False, )['affordability_ratio'].mean()
df.sort_values(by='affordability_ratio', ascending=True).head(20).style.background_gradient(cmap='PuBu_r')

,county_name,affordability_ratio
46,Sierra County,4.604956
8,Del Norte County,4.679333
18,Lassen County,4.900056
6,Colusa County,5.623878
16,Kings County,6.062874
52,Tehama County,6.622882
54,Tulare County,6.671068
11,Glenn County,6.681734
13,Imperial County,6.822660
10,Fresno County,6.891329


## San Diego County analysis
I'm currently located in San Diego county and was curious what the different metrics are for each city in the county.

In [32]:
# Get data for only San Diego county
df_sd = df_final[df_final['county_name']=='San Diego County']

# Agreegate the different metrics
s_sd_sai = df_sd.groupby('city_name')['school_affordability_index'].mean()
s_sd_ar = df_sd.groupby('city_name')['affordability_ratio'].mean()
s_sd_r = df_sd.groupby('city_name')['rating'].mean()
s_sd_hp = df_sd.groupby('city_name')['median_home_price'].mean()
s_sd_i = df_sd.groupby('city_name')['bachelors_degree'].mean()

# Create new dataframe with aggregated metrics and clean the data
df_sd_city = pd.concat([s_sd_sai, s_sd_ar, s_sd_r, s_sd_hp, s_sd_i], axis=1)
df_sd_city = df_sd_city[~df_sd_city['rating'].isna()]
df_sd_city.sort_values('school_affordability_index', ascending=False)

,school_affordability_index,affordability_ratio,rating,median_home_price,bachelors_degree
city_name,,,,,
Boulevard,1.42,4.92,7.00,"439,494.00","89,311.00"
Julian,0.58,9.23,5.33,"542,035.00","58,750.00"
Borrego Springs,0.57,7.03,4.00,"323,194.00","45,984.00"
Chula Vista,0.53,13.02,6.74,"802,874.69","62,326.06"
Santee,0.53,11.98,6.30,"768,210.00","64,124.00"
Bonsall,0.52,12.73,6.67,"969,675.00","76,195.00"
Carlsbad,0.48,16.92,7.77,"1,386,072.00","84,140.05"
Poway,0.48,14.92,7.20,"1,111,308.00","74,481.00"
La Mesa,0.45,13.26,6.00,"874,601.00","65,860.63"


## A different approach to the analysis
Comparing metrics across ZIP codes is one way to look at this data. I came to think that this method was too rigid and didn't reflect real world scenarios. For example you don't have to work in the same ZIP code that you live in. This means that a comparison of home prices and income at the ZIP code level is probably not the best way to analyze the data. Additionally, income and home price on a very local level (like ZIP code) will have high colliniarity. For example, expensive neighborhoods have expensive homes which require homeowners with a higher income to afford them. That leaves the question: at what level do we compare school ratings, income and housing prices?  
  
A "job market" is defined as a geographical area in which employers compete for employees. I will make the assumption that salaries & income are efficient within a job market and will define a job market as a county. Ex. San Diego county is one job market where employees can be expected to compete for jobs across the county, but not outside of it. 

The following two heuristics will drive the analysis:
1. You must live within school boundries to attend the school: I will compare home prices to school ratings at the lowest geographical level (ZIP code)
2. A reasonable commute would be to a job in the same county: compare income at the county level

The above heuristics will improve the target `school_affordability_index` metric as follows:  

`school_affordability_index = greatschools_rating_by_zip_code / (median_house_price_by_zip_code / median_income_by_county)`

In [35]:
# Get the mean school rating grouped by ZIP code
s_rating = df_final.groupby('zip_code')['rating'].mean()

# Get the mean home price grouped by ZIP code
s_home_price = df_final.groupby('zip_code')['median_home_price'].mean()

# Join the data grouped by ZIP code
df = pd.DataFrame({'median_home_price': s_home_price, 'rating': s_rating})

# Get the mean income grouped by county
s_income_county = df_final.groupby('county_name')['bachelors_degree'].mean()

# Join county and city names
df = df.merge(df_final[['county_name','city_name']], how='left', left_index=True, right_index=True)

# Join mean county income to the ZIP-level data
df = df.merge(s_income_county, left_on='county_name', right_index=True)

# Calculate the school_affordability_index, clean and show the data
df['school_affordability_index'] = df['rating'] / (df['median_home_price'] / df['bachelors_degree'])
df.drop_duplicates(inplace=True)
df.sort_values(by='school_affordability_index', ascending=False).head(50)

,median_home_price,rating,county_name,city_name,bachelors_degree,school_affordability_index
zip_code,,,,,,
92257,"57,862.00",6.00,Imperial County,Niland,"47,603.07",4.94
93562,"59,346.00",4.50,San Bernardino County,Trona,"51,779.73",3.93
96107,"310,064.00",7.50,Mono County,Coleville,"84,592.84",2.05
93523,"180,644.00",6.50,Kern County,Edwards,"56,603.40",2.04
96101,"149,762.00",4.67,Modoc County,Alturas,"63,842.00",1.99
93224,"171,064.00",6.00,Kern County,Fellows,"56,603.40",1.99
92398,"145,830.00",5.50,San Bernardino County,Yermo,"51,779.73",1.95
96134,"118,867.00",3.50,Modoc County,Tulelake,"63,842.00",1.88
93283,"156,865.00",5.00,Kern County,Weldon,"56,603.40",1.80
